# Problem 4.10

## part a

In [17]:
clear
clear mata
clear matrix
set matsize 11000 
set maxvar 32767
set seed 1234567

qui import delim ./data/chapter04/Arceneaux_AAAPSSsubset_2005, clear
set seed 1234567

In [18]:
rename treatment Z
rename vote03 Y
rename unit clust

global cov v_p2003 v_m2003 v_g2002 v_p2002 v_m2002 ///
 v_s2001 v_g2000 v_p2000 v_m2000 v_s1999 ///
 v_m1999 v_g1998 v_m1998 v_s1998 v_m1997 ///
 v_s1997 v_g1996 v_p1996 v_m1996 v_s1996

In [3]:
ritest Z Fstat=e(F), right clu(clust) reps(10000) nodots: reg Z $cov


      Source |       SS           df       MS      Number of obs   =     9,712
-------------+----------------------------------   F(20, 9691)     =      1.65
       Model |  8.23394346        20  .411697173   Prob > F        =    0.0341
    Residual |  2419.15557     9,691  .249629097   R-squared       =    0.0034
-------------+----------------------------------   Adj R-squared   =    0.0013
       Total |  2427.38952     9,711  .249962879   Root MSE        =    .49963

------------------------------------------------------------------------------
           Z |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
     v_p2003 |  -.0141049   .0179892    -0.78   0.433    -.0493674    .0211577
     v_m2003 |   -.028556    .013061    -2.19   0.029    -.0541582   -.0029538
     v_g2002 |  -.0191478   .0118595    -1.61   0.106    -.0423949    .0040992
     v_p2002 |   .0122656    .016042     0.76   0.

In [4]:
// one-tail p-value
di %8.3f el(r(p),1,1)

   0.939


## part b

In [19]:
// calculate the probabibility under cluster assignment
preserve
collapse Z, by(clust)   
qui sum Z
global p = r(mean)
restore

// probability of being assigned to treatment
gen probs = $p
sum probs









    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
       probs |      9,712          .5           0         .5         .5


In [6]:
//ate
qui reg Y Z $cov
di "ATE= " _b[Z]



ATE= .05596196


In [7]:
ritest Z ate_sim=_b[Z], right clu(clust) reps(10000) nodots: reg Y Z $cov


      Source |       SS           df       MS      Number of obs   =     9,712
-------------+----------------------------------   F(21, 9690)     =    220.91
       Model |  676.860032        21  32.2314301   Prob > F        =    0.0000
    Residual |  1413.81501     9,690  .145904542   R-squared       =    0.3238
-------------+----------------------------------   Adj R-squared   =    0.3223
       Total |  2090.67504     9,711  .215289367   Root MSE        =    .38197

------------------------------------------------------------------------------
           Y |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
           Z |    .055962   .0077661     7.21   0.000     .0407388    .0711851
     v_p2003 |   .2189582   .0137535    15.92   0.000     .1919985    .2459179
     v_m2003 |   .2313273   .0099878    23.16   0.000     .2117492    .2509054
     v_g2002 |    .110736    .009068    12.21   0.

In [8]:
// ate
di %8.5f el(r(b),1,1)

 0.05596


In [11]:
// one-tail p-value
di %8.4f el(r(p),1,1)

  0.0013


## part c

In [20]:
qui gen weights = Z/probs + (1 - Z)/(1 - probs)
qui tabstat Y [aw=weights], by(Z)  stat(sum) save

scalar ateHT = (el(r(Stat2),1,1) - el(r(Stat1),1,1))/_N
di "ate_HT ="%8.5f ateHT





ate_HT = 0.05395


## part d

In [21]:
cap program drop di_in_total
program define di_in_total, rclass
tabstat Y [aw=weights], by(Z)  stat(sum) save
return scalar ateHT_sim = (el(r(Stat2),1,1) - el(r(Stat1),1,1))/_N
end

ritest Z ateHT_sim=r(ateHT_sim), right clu(clust) reps(10000) nodots: di_in_total





Summary for variables: Y
     by categories of: Z 

       Z |       sum
---------+----------
       0 |      2784
       1 |      3308
---------+----------
   Total |      6092
--------------------

      command:  di_in_total
    ateHT_sim:  r(ateHT_sim)
  res. var(s):  Z
   Resampling:  Permuting Z
Clust. var(s):  clust
     Clusters:  28
Strata var(s):  none
       Strata:  1

------------------------------------------------------------------------------
T            |     T(obs)       c       n   p=c/n   SE(p) [95% Conf. Interval]
-------------+----------------------------------------------------------------
   ateHT_sim |   .0539539    1954   10000  0.1954  0.0040  .1876681   .2033097
------------------------------------------------------------------------------
Note: Confidence interval is with respect to p=c/n.
Note: c = #{T >= T(obs)}


In [22]:
// one-tail p-value
di %8.3f el(r(p),1,1)

   0.195


## part e

In [23]:
mkmat $cov , mat(cov_m)
matrix rowm = cov_m * J(colsof(cov_m), 1, 1/colsof(cov_m))
matrix colnames rowm=row_mean
svmat double rowm, names(col)

qui gen Y_diff = Y - row_mean

In [24]:
cap program drop di_in_total2
program define di_in_total2, rclass
tabstat Y_diff [aw=weights], by(Z)  stat(sum) save
return scalar ateHT_sim2 = (el(r(Stat2),1,1) - el(r(Stat1),1,1))/_N
end

ritest Z ateHT_sim=r(ateHT_sim2), right clu(clust) reps(10000) nodots: di_in_total2





Summary for variables: Y_diff
     by categories of: Z 

       Z |       sum
---------+----------
       0 | -38.40001
       1 |       435
---------+----------
   Total |     396.6
--------------------

      command:  di_in_total2
    ateHT_sim:  r(ateHT_sim2)
  res. var(s):  Z
   Resampling:  Permuting Z
Clust. var(s):  clust
     Clusters:  28
Strata var(s):  none
       Strata:  1

------------------------------------------------------------------------------
T            |     T(obs)       c       n   p=c/n   SE(p) [95% Conf. Interval]
-------------+----------------------------------------------------------------
   ateHT_sim |   .0487438     110   10000  0.0110  0.0010  .0090491    .013243
------------------------------------------------------------------------------
Note: Confidence interval is with respect to p=c/n.
Note: c = #{T >= T(obs)}


In [25]:
// ateHT2
di %8.5f el(r(b),1,1)

 0.04874


In [26]:
// one-tail p-value
di %8.3f el(r(p),1,1)

   0.011
